In [1]:
!pip install -q ragstack-ai datasets

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() #to laod the required api's ASTRA_DB_APPLICATION_TOKEN,OPENAI_API_KEY,ASTRA_DB_API_ENDPOINT

True

In [3]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings

In [4]:
embedding_func =OpenAIEmbeddings()

In [5]:
vstore = AstraDBVectorStore(collection_name="test",embedding= embedding_func,
                            token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
                           api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"))
print("AsraDB Vector Storeis configured")

AsraDB Vector Storeis configured


In [6]:
!pip install datasets

In [7]:
from datasets import load_dataset
philo_dataset=load_dataset("datastax/philosopher-quotes")["train"]
philo_dataset[16]

C:\Users\ankit\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'author': 'aristotle',
 'quote': 'Love well, be loved and do something of value.',
 'tags': 'love;ethics'}

In [8]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader
dataset_name = "datastax/philosopher-quotes"
page_content_column = "quote"
loader=HuggingFaceDatasetLoader(dataset_name,page_content_column)
data = loader.load()

C:\Users\ankit\anaconda3\lib\site-packages\datasets\load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [9]:
data

[Document(page_content='"True happiness comes from gaining insight and growing into your best possible self. Otherwise all you\'re having is immediate gratification pleasure, which is fleeting and doesn\'t grow you as a person."', metadata={'author': 'aristotle', 'tags': 'knowledge'}),
 Document(page_content='"The roots of education are bitter, but the fruit is sweet."', metadata={'author': 'aristotle', 'tags': 'education;knowledge'}),
 Document(page_content='"Before you heal the body you must first heal the mind"', metadata={'author': 'aristotle', 'tags': 'ethics'}),
 Document(page_content='"The proof that you know something is that you are able to teach it"', metadata={'author': 'aristotle', 'tags': 'education;knowledge'}),
 Document(page_content='"Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons."', metadata={'author': 'aristotle', 'tags': None}),
 

In [10]:
inserted_data=vstore.add_documents(data)

In [11]:
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '3a942bb31ed9424785c00861bd73be5a', 'content': '"We learn from history that man can never learn anything from history."', '$vector': [-0.005888790454996967, -0.008994692373649119, 0.019753536835926996, -0.02211402404498896, 0.007565977509695579, -0.013392649684175648, -0.010013428821365175, 0.017728489645507797, -0.009796015305217292, 0.000969817917638462, 0.01634946921970327, 0.027431327995611002, 0.037196283694908586, -0.012578903841562107, -0.0020421306721669238, 0.011957722899038167, 0.029468799028398083, -0.010007216900181233, 0.020809544810746698, -0.023182453999531515, -0.01381505287410353, 0.017094884860615975, 0.022561273057007575, -0.0455697969886788, 0.009112717013498972, -0.015169226546494805, 0.009280435160175323, -0.04211603214033851, 0.020511378181852613, -0.04196694975721398, 0.0064540645725266925, 0.013554156840990575, 0.010237053327374483, -0.05051439311884449, -0.022797322336707282, -0.012554056156826343, -0.007621883869028535, -0.0098

### Basic Retrieval

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [13]:
retriever =vstore.as_retriever(search_kwargs={'k':3})

In [14]:
prompt_template = """
Answer the question based only on the supllied context.If you don't know the answer,say you do not know the answer
Context:{context}
Question:{question}
Your answer:
"""

prompt=ChatPromptTemplate.from_template(prompt_template)
model=ChatOpenAI()
chain= ({'context':retriever,"question":RunnablePassthrough()}
       | prompt
       | model 
       | StrOutputParser()
    )

C:\Users\ankit\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
print(chain.invoke("IN the given context, what subject are philoshper more concerned with?Also provide the document text and meta dataas a refrence"))

In the given context, philosophers are more concerned with the subject of knowledge. 

Document 1:
Text: "The philosopher is in love with truth, that is, not with the changing world of sensation, which is the object of opinion, but with the unchanging reality which is the object of knowledge."
Metadata: Author - Plato, Tags - Knowledge

Document 2:
Text: "Philosophy stands in need of a science which shall determine the possibility, principles, and extent of human knowledge priori."
Metadata: Author - Kant, Tags - Knowledge

Document 3:
Text: "All the interests of my reason, speculative as well as practical, combine in the three following questions: 1. What can I know? 2. What ought I to do? 3. What may I hope?"
Metadata: Author - Kant, Tags - Knowledge


In [28]:
print(chain.invoke("What IS THE IMPORTANCE TO ALLOW THE BRAIN ?ALSO PROVIDE THE REFRENCE TEXT"))

The importance is to allow the brain the full measure of sleep which is required to restore it. The reference text is: "It is most important to allow the brain the full measure of sleep which is required to restore it; for sleep is to a man's whole nature what winding up is to a clock." (Schopenhauer)
